In [1]:
import pandas as pd
import plotly.express as px

In [2]:
skipped_lines = []

# Custom function to handle bad lines
def bad_line_handler(line):
    skipped_lines.append(line)
    return None 

In [11]:
filename = '../../data/Static.csv'
df = pd.read_csv(filename, 
                 encoding='euc-kr', 
                 on_bad_lines=bad_line_handler,
                 engine='python')
                 
kor_columns = list(df.columns)
eng_columns = ["MMSI", "Ship Name", "Linetype Code", "IMO", "Call Sign", "DimA", "DimB", "DimC", "DimD", "Draft", "Estimated Tons"]
df.columns = eng_columns
print(f"Korean column names: {kor_columns}")
print(f"English column names: {eng_columns}")
print(df.head())

Korean column names: ['MMSI', '선박명', '선종코드', 'IMO', '호출부호', 'DimA', 'DimB', 'DimC', 'DimD', '흘수', '추정톤수']
English column names: ['MMSI', 'Ship Name', 'Linetype Code', 'IMO', 'Call Sign', 'DimA', 'DimB', 'DimC', 'DimD', 'Draft', 'Estimated Tons']
   MMSI      Ship Name  Linetype Code          IMO Call Sign  DimA  DimB  \
0     0            NaN            0.0          NaN       NaN   0.0   0.0   
1     1  HEMINGWAY3600           52.0  910417200.0      V3GU  13.0  20.0   
2    10   BAOLI-10-99%            0.0          0.0       NaN   0.0   0.0   
3   100    SUQIYU01201           30.0          0.0       600  26.0   8.0   
4  1000              0           30.0        100.0       AAA  23.0  15.0   

   DimC  DimD  Draft  Estimated Tons  
0   0.0   0.0    0.0             0.0  
1   6.0   3.0    4.0           114.0  
2   0.0   0.0    0.0             0.0  
3   5.0   2.0    0.0           124.0  
4   3.0   3.0    0.0           174.0  


In [14]:
len(skipped_lines)/len(df)

0.0004260467969801803

58679

In [3]:
filename = '../../raw_data/Dynamic_20230501.csv'
df_dynamic = pd.read_csv(filename, 
                 encoding='euc-kr', 
                 skiprows=2)
# , 
#                  on_bad_lines=bad_line_handler,
                #  engine='python')s
                 
kor_columns = list(df_dynamic.columns)
eng_columns = ["MMSI", "Date", "Latitude", "Longitude", "SOG", "COG", "Heading"]
# SOG = speed over ground
# COG = course over ground
# heading = where the ship is pointing
df_dynamic.columns = eng_columns
print(f"Korean column names: {kor_columns}")
print(f"English column names: {eng_columns}")
print(df_dynamic.head())

Korean column names: ['MMSI', '일시', '위도', '경도', 'SOG', 'COG', 'Heading']
English column names: ['MMSI', 'Date', 'Latitude', 'Longitude', 'SOG', 'COG', 'Heading']
        MMSI                 Date   Latitude   Longitude   SOG    COG  Heading
0          0  2023-05-01 06:39:40  36.701558  125.519965   1.7  244.6      281
1  100000000  2023-05-01 09:04:23  36.717567  122.198050  10.1  342.6      511
2  100001200  2023-05-01 00:00:20  32.241408  125.398798   0.0  166.0      511
3  100001200  2023-05-01 00:01:33  32.241402  125.398590   0.1  267.8      511
4  100001200  2023-05-01 00:03:52  32.241368  125.398648   0.0  124.0      511


In [4]:
len(df_dynamic)

5354032

In [5]:
data_grouped = df_dynamic.groupby("MMSI")

In [ ]:
for mmsi, data in data_grouped:
    if len(data) < 10:
        continue
    # print(mmsi, len(data))
    # print()
    
    data.to_csv(f"../../processed_data/mmsi_{mmsi}_len_{len(data)}.csv")
    # break

23 10
233 18
14347 11
27347 22
27556 44
32556 16
125088 19
4121026 16
21106868 34
41200005 19
62288005 635
62288009 634
62288022 662
62288023 672
62288025 633
62288035 612
62288042 612
62288047 655
66225803 599
66225804 626
66225805 186
66225807 164
66225813 652
66225814 639
66228809 609
66228810 615
66228811 612
66228812 662
66228813 638
66228814 671
66596661 24
66853534 19
96048304 111
96052585 89
96987979 66
100001200 85
100044559 46
102890206 129
104008821 83
108000858 11
108001333 15
108672075 14
121990017 18
150000730 15
150800006 10
156290001 81
166095012 49
166095019 15
199009165 99
200000000 57
200377613 24
200731005 12
209795000 1410
210616000 145
210881197 17
210970000 603
211828850 1801
211882930 2272
213089778 32
215865000 511
218427000 3168
219045000 73
221005409 23
222200082 70
223581303 16
224560023 18
229075000 10326
229132000 1859
229334000 913
229488000 2326
229876000 1446
229910000 2279
232006301 13174
232007360 5588
232008054 2356
232046382 10090
234567890 880
2350

In [4]:
df_dynamic['Date'] = pd.to_datetime(df_dynamic['Date'])
df_dynamic = df_dynamic.sort_values(by=['MMSI', 'Date'])

In [5]:
df_to_plot = df_dynamic[df_dynamic['MMSI']< 100000000]

In [6]:
len(df_to_plot)/len(df_dynamic)

0.002318439635773563

In [7]:
# fig = px.scatter_mapbox(
#     df_to_plot,
#     lat="Latitude",
#     lon="Longitude",
#     color="MMSI",  # Different colors for different ships
#     size="SOG",    # Size of points based on speed
#     animation_frame="Date",  # Dynamic animation over time
#     title="Dynamic Ship Trajectories",
#     hover_data=["COG", "Heading"]
# )

# # Configure the map style
# fig.update_layout(
#     mapbox_style="carto-positron",
#     mapbox_zoom=10,  # Adjust the zoom level
#     mapbox_center={"lat": df_to_plot["Latitude"].mean(), "lon": df_to_plot["Longitude"].mean()},
#     height=600,
# )

fig = px.line_mapbox(
    df_to_plot,
    lat="Latitude",
    lon="Longitude",
    color="MMSI",  # Different colors for different boats
    title="Ship Trajectories (1-5 Boats)",
    hover_data=["Date", "SOG", "COG", "Heading"]
)

# Configure the map style
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10,  # Adjust the zoom level
    mapbox_center={"lat": df_to_plot["Latitude"].mean(), "lon": df_to_plot["Longitude"].mean()},
    height=600,
)

fig.write_html("ship_trajectories.html")


In [ ]:
# fig.write_html("ship_trajectories_dynamic.html")